# importing libraries and data reading

In [1]:
import pandas as pd
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)
import re
import ast
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import warnings
warnings.filterwarnings("ignore")

In [105]:
df = pd.read_excel('zillow_full.xlsx')

In [106]:
a=['countyId','zipcode', 'zestimate','lotSize',
       'rentZestimate','bedrooms', 'bathrooms', 'livingAreaValue', 
       'lastSoldPrice', 'zestimateLowPercent', 'zestimateHighPercent',
       'restimateLowPercent', 'restimateHighPercent', 
       'taxAssessedValue', 'taxAssessedYear', 
       'propertyTaxRate',  
       'resoFacts_hasFireplace', 'resoFacts_parking',
       'mortgageRates_thirtyYearFixedRate',
       'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate', 
    'city', 'county' , 'description','parentRegion_name',
    'dateSoldString', 'resoFacts_daysOnZillow', 'resoFacts_flooring', 'resoFacts_hasAttachedProperty',
    'resoFacts_hasGarage', 'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa' , 'resoFacts_hasView','resoFacts_hoaFee',
     'resoFacts_pricePerSquareFoot','resoFacts_yearBuilt' ,'solarPotential_sunScore','schools'
   ]

In [107]:
df=df[a]

In [108]:
df.shape

(1325, 38)

In [109]:
df.isnull().sum().sum() # total null values in our dataset

4380

# Data Cleaning and Null Values removal

In [110]:
df['resoFacts_hoaFee'].fillna(df['resoFacts_hoaFee'].mode()[0], inplace=True)
hoe=[]
for x in df['resoFacts_hoaFee']:
    x=str(x).replace(",","")
    hoe.append(int(re.findall('[0-9,]+', x)[0]))
df['resoFacts_hoaFee']=hoe   

In [111]:
df['resoFacts_flooring'].fillna(df['resoFacts_flooring'].mode()[0], inplace=True)
flooring=[]
for x in df['resoFacts_flooring']:
    x=x.replace('[',"")
    flooring.append(x.replace(']',""))
df['resoFacts_flooring']= flooring

In [112]:
df_cat= df.select_dtypes(exclude='number')
print(df_cat.shape)

(1325, 7)


In [113]:
print(df_cat.isnull().sum())

city                    0
county                  0
description             3
parentRegion_name       2
dateSoldString        392
resoFacts_flooring      0
schools                 3
dtype: int64


In [114]:
for column in df_cat:
    df_cat[column].fillna(df_cat[column].mode()[0], inplace=True)

In [115]:
discrete_cols= ["bedrooms", "bathrooms", "resoFacts_hasFireplace", "resoFacts_parking", "resoFacts_daysOnZillow" ,"resoFacts_hasAttachedProperty" , "resoFacts_hasGarage" , "resoFacts_hasPetsAllowed", "resoFacts_hasSpa", "resoFacts_hasView", "resoFacts_yearBuilt"]

In [116]:
numeric_cols=["lotSize","zestimate","rentZestimate", "livingAreaValue", "lastSoldPrice", "zestimateLowPercent", "zestimateHighPercent", "restimateLowPercent" ,"restimateHighPercent" , "taxAssessedValue" , "propertyTaxRate", "resoFacts_hoaFee", "resoFacts_pricePerSquareFoot" , "solarPotential_sunScore"]

In [117]:
df_discrete=df[discrete_cols]
df_num=df[numeric_cols]

In [118]:
df_num.isnull().sum()

lotSize                         120
zestimate                       252
rentZestimate                   164
livingAreaValue                  23
lastSoldPrice                   392
zestimateLowPercent             252
zestimateHighPercent            252
restimateLowPercent             300
restimateHighPercent            300
taxAssessedValue                206
propertyTaxRate                   2
resoFacts_hoaFee                  0
resoFacts_pricePerSquareFoot     53
solarPotential_sunScore         219
dtype: int64

In [119]:
from sklearn.impute import KNNImputer   # missing values imputation

imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_num.iloc[:,0:]=imputer.fit_transform(df_num.iloc[:,0:]) #replace df with imputed missing values

In [120]:
df_num.isnull().sum().sum()

0

In [121]:
df_discrete.isnull().sum()

bedrooms                          49
bathrooms                         24
resoFacts_hasFireplace           223
resoFacts_parking                  0
resoFacts_daysOnZillow             2
resoFacts_hasAttachedProperty      0
resoFacts_hasGarage                0
resoFacts_hasPetsAllowed           0
resoFacts_hasSpa                   0
resoFacts_hasView                  0
resoFacts_yearBuilt               20
dtype: int64

In [122]:
#df_discret= df_discrete.dropna()

In [123]:
df_discrete.resoFacts_hasFireplace.value_counts()

1.00000    734
0.00000    368
Name: resoFacts_hasFireplace, dtype: int64

In [124]:
df_discrete['resoFacts_hasFireplace'].fillna(0, inplace=True)

In [125]:
df_discrete.isnull().sum()

bedrooms                         49
bathrooms                        24
resoFacts_hasFireplace            0
resoFacts_parking                 0
resoFacts_daysOnZillow            2
resoFacts_hasAttachedProperty     0
resoFacts_hasGarage               0
resoFacts_hasPetsAllowed          0
resoFacts_hasSpa                  0
resoFacts_hasView                 0
resoFacts_yearBuilt              20
dtype: int64

In [126]:
for column in df_discrete:
    df_discrete[column].fillna(df_discrete[column].mode()[0], inplace=True)

In [127]:
df_discrete.isnull().sum()

bedrooms                         0
bathrooms                        0
resoFacts_hasFireplace           0
resoFacts_parking                0
resoFacts_daysOnZillow           0
resoFacts_hasAttachedProperty    0
resoFacts_hasGarage              0
resoFacts_hasPetsAllowed         0
resoFacts_hasSpa                 0
resoFacts_hasView                0
resoFacts_yearBuilt              0
dtype: int64

In [128]:
print(df_cat.shape)
print(df_num.shape)
print(df_discrete.shape)

(1325, 7)
(1325, 14)
(1325, 11)


In [129]:
df_discrete['resoFacts_parking']=np.where(df_discrete['resoFacts_parking']>=1,1,0) 


In [130]:
df_discrete.resoFacts_parking.value_counts()

1    768
0    557
Name: resoFacts_parking, dtype: int64

In [131]:
df= pd.concat([df_cat,df_num,df_discrete], axis=1)  # this df is without null values

In [132]:
df.shape

(1325, 32)

In [133]:
df.isnull().sum().sum()

0

# Feature Engineering- Creating 6 features out of school

In [134]:
#df_2= pd.read_excel('zillow_full.xlsx')

In [135]:
schools= list(df['schools'])

In [136]:
list_schools= [[]] * len(schools)

In [137]:
len(list_schools)

1325

In [138]:
i=0
for x in schools:
    list_schools[i]=[0,0,0,0,0,0]
    x = str(x).replace('\\','"')
    l=re.findall("[{][\\a-zA-z0-9:',\-/ \.=?&]+[}]", x)
    for y in l:
        y=ast.literal_eval(y)
        if y['level']== 'Elementary':
            if y['rating']!=None:
                if list_schools[i][1]<y['rating']:
                    list_schools[i][1]=y['rating']
                    list_schools[i][0]=y['distance']
                
        if y['level']== 'Middle':
            if y['rating']!=None:
                if list_schools[i][3]<y['rating']:
                    list_schools[i][3]=y['rating']
                    list_schools[i][2]=y['distance']
        if y['level']=='High':
            if y['rating']!=None:
                if list_schools[i][5]<y['rating']:
                    list_schools[i][5]=y['rating']
                    list_schools[i][4]=y['distance']
    
    i+=1
    
    

In [139]:
list_schools[0]

[0.7, 3, 0, 0, 0, 0]

In [140]:
for i in range(len(list_schools)):
    list_schools[i] = [None if item <=0 else item for item in list_schools[i]]

In [141]:
df_3 = pd.DataFrame(list_schools, columns = ['Elementary_Distance', 'Elementary_Ratings', 'Middle_Distance', 'Middle_Ratings', 'High_Distance', 'High_Ratings'])

In [142]:
df_3['Elementary_Distance']=np.where(np.isnan(df_3['Elementary_Distance']),df_3['Elementary_Distance'].median(),df_3['Elementary_Distance']) 
df_3['Middle_Ratings']=np.where(np.isnan(df_3['Middle_Ratings']),df_3['Middle_Ratings'].mode(),df_3['Middle_Ratings']) 
df_3['Middle_Distance']=np.where(np.isnan(df_3['Middle_Distance']),df_3['Middle_Distance'].median(),df_3['Middle_Distance']) 
df_3['High_Ratings']=np.where(np.isnan(df_3['High_Ratings']),df_3['High_Ratings'].mode(),df_3['High_Ratings']) 
df_3['High_Distance']=np.where(np.isnan(df_3['High_Distance']),df_3['High_Distance'].median(),df_3['High_Distance']) 


In [143]:
imputer = KNNImputer(n_neighbors=2)  #impute missing values
df_3.iloc[:,0:]=imputer.fit_transform(df_3.iloc[:,0:]) #replace df with imputed missing values

In [144]:
df_3.shape

(1325, 6)

In [145]:
df_3.isnull().sum()

Elementary_Distance    0
Elementary_Ratings     0
Middle_Distance        0
Middle_Ratings         0
High_Distance          0
High_Ratings           0
dtype: int64

In [146]:
df.shape

(1325, 32)

In [147]:
df.drop('schools', axis=1, inplace= True)

In [148]:
df.shape

(1325, 31)

In [149]:
df=df.reset_index()

In [150]:
df=pd.concat([df,df_3], axis=1)

In [151]:
df.shape  # this df contains school columns also

(1325, 38)

In [152]:
df.isnull().sum().sum()

0

# Outliers

In [153]:
#columns_for_outliers=[ 'bedrooms','lastSoldPrice','Elementary_Ratings','Elementary_Distance',
 #      'Middle_Ratings','Middle_Distance', 'High_Distance', 'High_Ratings','livingAreaValue','resoFacts_hasGarage']

In [154]:
columns_for_outliers= ['Elementary_Ratings','Elementary_Distance',
    'Middle_Ratings','Middle_Distance', 'High_Distance', 'High_Ratings']

In [155]:
# Detecting outliers and imputing nan 
from scipy.stats import skew

for x in columns_for_outliers:
        percentile25 = df[x].quantile(0.25)
        percentile75 = df[x].quantile(0.75)
        iqr= percentile75-percentile25
        upper = percentile75 + 1.5 * iqr
        lower = percentile25 - 1.5 * iqr
        for y in df[x]:
            if (y>upper) or (y<lower):
                df[x].loc[df[x] == y]=None
    
    
       

In [156]:
df.isnull().sum()

index                              0
city                               0
county                             0
description                        0
parentRegion_name                  0
dateSoldString                     0
resoFacts_flooring                 0
lotSize                            0
zestimate                          0
rentZestimate                      0
livingAreaValue                    0
lastSoldPrice                      0
zestimateLowPercent                0
zestimateHighPercent               0
restimateLowPercent                0
restimateHighPercent               0
taxAssessedValue                   0
propertyTaxRate                    0
resoFacts_hoaFee                   0
resoFacts_pricePerSquareFoot       0
solarPotential_sunScore            0
bedrooms                           0
bathrooms                          0
resoFacts_hasFireplace             0
resoFacts_parking                  0
resoFacts_daysOnZillow             0
resoFacts_hasAttachedProperty      0
r

In [157]:
df= df.dropna()

In [158]:
df.shape

(1052, 38)

# Feature Engineering- Creating New feature out of feature("description")

In [159]:
sentiment_des = []
from textblob import TextBlob
for text in df['description'].values:
        t= TextBlob(text)
        sentiment_des.append(t.sentiment[0])
df['desc_sentiment']=sentiment_des

In [160]:
df['desc_sentiment']=sentiment_des

In [161]:
df.drop(['description'], axis=1,inplace=True)

In [162]:
df.shape 

(1052, 38)

In [163]:
df.corr()

,index,lotSize,zestimate,rentZestimate,livingAreaValue,lastSoldPrice,zestimateLowPercent,zestimateHighPercent,restimateLowPercent,restimateHighPercent,taxAssessedValue,propertyTaxRate,resoFacts_hoaFee,resoFacts_pricePerSquareFoot,solarPotential_sunScore,bedrooms,bathrooms,resoFacts_hasFireplace,resoFacts_parking,resoFacts_daysOnZillow,resoFacts_hasAttachedProperty,resoFacts_hasGarage,resoFacts_hasPetsAllowed,resoFacts_hasSpa,resoFacts_hasView,resoFacts_yearBuilt,Elementary_Distance,Elementary_Ratings,Middle_Distance,Middle_Ratings,High_Distance,High_Ratings,desc_sentiment
index,1.00000,0.02689,-0.03106,-0.03259,-0.01835,-0.07015,0.01282,0.02277,-0.03855,-0.07335,-0.04710,0.13955,-0.10777,0.01299,-0.01230,0.01082,-0.00981,0.03774,-0.00496,0.02043,-0.08774,-0.00956,-0.04461,0.03784,-0.02259,0.03526,0.03878,-0.04695,0.05177,-0.00980,0.00846,-0.00880,-0.03439
lotSize,0.02689,1.00000,-0.04502,-0.03515,-0.03748,-0.04330,-0.02402,-0.01446,-0.02617,-0.02505,-0.03302,0.01226,0.01134,-0.00184,0.02237,-0.04538,-0.04683,-0.05111,-0.01787,-0.00439,-0.01477,-0.00841,0.03180,-0.01156,-0.02566,0.00991,0.06275,0.02978,0.00120,-0.04589,-0.03614,-0.01257,-0.02118
zestimate,-0.03106,-0.04502,1.00000,0.80052,0.61576,0.53075,0.11156,0.12777,0.26625,0.24130,0.51761,-0.32333,0.16772,0.01343,-0.03543,0.42354,0.45380,0.15024,0.08798,0.03854,0.07118,0.10597,-0.04328,-0.00916,0.17854,-0.00627,-0.04679,0.32523,-0.02264,0.35686,-0.00983,0.21269,-0.00653
rentZestimate,-0.03259,-0.03515,0.80052,1.00000,0.45606,0.45223,0.14387,0.16135,0.30748,0.24018,0.47781,-0.36306,0.12307,-0.02509,-0.05665,0.34285,0.42420,0.08203,0.13633,-0.00225,0.14664,0.15406,-0.00926,-0.00509,0.19201,-0.08459,-0.03306,0.23535,-0.04552,0.22872,-0.07220,0.10026,0.00197
livingAreaValue,-0.01835,-0.03748,0.61576,0.45606,1.00000,0.44994,0.17669,0.24243,0.24750,0.27842,0.37761,-0.09896,-0.04669,-0.05327,-0.02605,0.62198,0.56129,0.11342,0.11561,-0.00030,-0.12627,0.15997,-0.09813,0.03828,0.12155,0.09472,-0.00663,0.12965,0.01207,0.19142,0.02124,0.12816,-0.02667
lastSoldPrice,-0.07015,-0.04330,0.53075,0.45223,0.44994,1.00000,0.14717,0.19664,0.23385,0.29968,0.64661,-0.34896,0.08970,-0.02311,-0.02147,0.21555,0.32947,-0.00782,0.10933,-0.00303,0.13100,0.11742,0.02172,-0.05261,0.19343,-0.06074,-0.00209,0.15413,-0.02254,0.17554,-0.07347,0.03752,0.02524
zestimateLowPercent,0.01282,-0.02402,0.11156,0.14387,0.17669,0.14717,1.00000,0.87128,0.18070,0.25283,0.03104,-0.15836,0.00052,-0.00305,-0.12137,0.07188,-0.02383,-0.07879,0.08221,-0.05539,0.08319,0.13506,0.03108,-0.07775,0.00879,-0.06319,-0.08072,-0.04509,-0.10960,-0.02944,-0.16744,-0.07241,0.01941
zestimateHighPercent,0.02277,-0.01446,0.12777,0.16135,0.24243,0.19664,0.87128,1.00000,0.21470,0.33002,0.07626,-0.13516,0.00278,-0.00606,-0.10298,0.10826,0.00194,-0.09325,0.07111,-0.05091,0.04837,0.11297,0.03517,-0.08546,0.03334,-0.08703,-0.05707,-0.05429,-0.11297,-0.06371,-0.15849,-0.09680,0.01393
restimateLowPercent,-0.03855,-0.02617,0.26625,0.30748,0.24750,0.23385,0.18070,0.21470,1.00000,0.44820,0.21614,-0.19833,0.02606,-0.03161,-0.03952,0.16319,0.13674,-0.04168,0.02160,-0.03398,0.09441,0.03753,0.01855,0.03973,0.08059,-0.04227,-0.04100,0.05911,-0.02542,0.04628,-0.06787,-0.00070,-0.01132
restimateHighPercent,-0.07335,-0.02505,0.24130,0.24018,0.27842,0.29968,0.25283,0.33002,0.44820,1.00000,0.23601,-0.29572,0.16632,-0.01158,-0.00975,0.08383,0.06071,-0.18951,0.15176,-0.03965,0.14267,0.13130,0.11388,-0.05187,0.08205,-0.10901,-0.04979,-0.00847,-0.05051,0.00145,-0.12162,-0.03917,-0.00919


# Train-Test Split

In [164]:
y = df.zestimate

In [165]:
x= df.drop('zestimate', axis=1)

In [166]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2, random_state=123)

In [167]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(841, 37)
(841,)
(211, 37)
(211,)


# Numeric Data Preparation and dealing with multicollinearity

In [168]:
from scipy.sparse import coo_matrix
from scipy.sparse import hstack


X_train_num_processed = coo_matrix((841,0))
X_test_num_processed = coo_matrix((211,0))

In [169]:
num_cols= ['lotSize', 'livingAreaValue',
       'lastSoldPrice', 
       'taxAssessedValue',
       'resoFacts_hoaFee', 'resoFacts_pricePerSquareFoot',
           'resoFacts_hasFireplace', 'resoFacts_parking', 'resoFacts_daysOnZillow',
       'resoFacts_hasAttachedProperty', 'resoFacts_hasGarage',
       'resoFacts_hasPetsAllowed', 'resoFacts_hasSpa', 'resoFacts_hasView',
       'Elementary_Distance', 
       'Middle_Distance',  'High_Distance'
       ]

In [170]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = num_cols
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [171]:
m= df.loc[:,num_cols]
calc_vif(m)

,variables,VIF
0,lotSize,1.01484
1,livingAreaValue,5.00166
2,lastSoldPrice,4.89050
3,taxAssessedValue,3.96587
4,resoFacts_hoaFee,2.96346
5,resoFacts_pricePerSquareFoot,1.03619
6,resoFacts_hasFireplace,2.45618
7,resoFacts_parking,5.18776
8,resoFacts_daysOnZillow,1.06578
9,resoFacts_hasAttachedProperty,1.57254


In [172]:
for x in num_cols:
    

    s_train = (X_train[x].values.reshape(-1, 1))

    s_test = (X_test[x].values.reshape(-1, 1))

    
    X_train_num_processed= hstack((X_train_num_processed,s_train)).tocsr()
    X_test_num_processed= hstack((X_test_num_processed,s_test)).tocsr()


In [173]:
print(X_train_num_processed.shape)
print(X_test_num_processed.shape)

(841, 17)
(211, 17)


# OLS regresssion 

In [174]:
X_train_test_processed=pd.concat([pd.DataFrame(X_train_num_processed.todense()),pd.DataFrame(X_test_num_processed.todense())],ignore_index=True)
Y_train_test_processed = pd.concat([pd.DataFrame(Y_train),pd.DataFrame(Y_test)], ignore_index=True)

In [175]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


X2 = sm.add_constant(X_train_test_processed)
est = sm.OLS(Y_train_test_processed, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              zestimate   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     73.31
Date:                Fri, 16 Jun 2023   Prob (F-statistic):          5.05e-164
Time:                        19:20:57   Log-Likelihood:                -15428.
No. Observations:                1052   AIC:                         3.089e+04
Df Residuals:                    1034   BIC:                         3.098e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.45e+05   7.13e+04      2.033      0.0